In [20]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [21]:
import numpy as np
import keras

In [22]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.models import Model

In [23]:
tmp = np.load('C:/Users/ISE/Desktop/Deep Learning/cifar_pca_train.npz')
train_data = tmp['data']
train_labels = tmp['labels']

In [24]:
tmp = np.load('C:/Users/ISE/Desktop/Deep Learning/cifar_pca_test.npz')
test_data = tmp['data']
test_labels = tmp['labels']

In [25]:
# Training an autoencoder model on cifar-10 PCA reduced data

input_img = Input(shape=(781,))
crrpt_img = Dropout(0.5)(input_img)
encoded = Dense(1000, activation='sigmoid')(crrpt_img)
decoded = Dense(781, activation='linear')(encoded)

In [ ]:
autoencoder = Model(input_img,decoded)

nb_epoch = 10
batch_size = 32

autoencoder.compile(optimizer='adam',
                    loss='mean_squared_error',
                    metrics=['mse'])

In [ ]:
history = autoencoder.fit(train_data, train_data,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(test_data, test_data),
                    verbose=1)


autoencoder.save('DAE_l1_model.h5')

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 76s - loss: 0.5828 - mean_squared_error: 0.5828 - val_loss: 0.2319 - val_mean_squared_error: 0.2319
Epoch 2/10
50000/50000 [==============================] - 75s - loss: 0.5373 - mean_squared_error: 0.5373 - val_loss: 0.2121 - val_mean_squared_error: 0.2121
Epoch 3/10
50000/50000 [==============================] - 73s - loss: 0.5284 - mean_squared_error: 0.5284 - val_loss: 0.2023 - val_mean_squared_error: 0.2023
Epoch 5/10
50000/50000 [==============================] - 84s - loss: 0.5251 - mean_squared_error: 0.5251 - val_loss: 0.2003 - val_mean_squared_error: 0.2003
Epoch 6/10
43328/50000 [========================>.....] - ETA: 10s - loss: 0.5241 - mean_squared_error: 0.5241

In [ ]:
encoder = Model(input_img,encoded)
htrain_data = encoder.predict(train_data)
htest_data = encoder.predict(test_data)

In [ ]:
# Converting labels into one-hot vectors for MLP training

train_labels = keras.utils.to_categorical(train_labels, 10)
test_labels = keras.utils.to_categorical(test_labels, 10)

In [ ]:
# training a two hidden layer MLP for classification task

mlp = Sequential()
mlp.add(Dropout(0.2, input_shape=(781,)))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(10, activation='softmax'))



mlp.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


history = mlp.fit(train_data[:10000], train_labels[:10000],
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(test_data, test_labels))

In [ ]:
# training an MLP on autoencoder features

mlp = Sequential()
mlp.add(Dropout(0.2, input_shape=(1000,)))
mlp.add(Dense(1000, activation='sigmoid'))
mlp.add(Dropout(0.5))
mlp.add(Dense(10, activation='softmax'))



mlp.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


history = mlp.fit(htrain_data[:10000], train_labels[:10000],
                    batch_size=batch_size,
                    epochs=nb_epoch,
                    verbose=1,
                    validation_data=(htest_data, test_labels))